# भाग 2: स्थानीय रूप से ग्रिड नेटवर्क लॉन्च करें

इस ट्यूटोरियल में, आप सीखेंगे कि ग्रिड मशीन को स्थानीय मशीन में कैसे तैनात किया जाए और फिर PySyft का उपयोग करके इसके साथ सहभागिता करें।

_चेतावनी: ग्रिड नोड ऑनलाइन डेटासेट प्रकाशित करते हैं और केवल प्रायोगिक उपयोग के लिए होते हैं। अपने जोखिम पर नोड्स तैनात करें। किसी भी डेटा / मॉडल के साथ OpenGrid का उपयोग न करें जिसे आप निजी रखना चाहते हैं।_


ग्रिड नेटवर्क चलाने के लिए स्थानीय स्तर पर आपको दो अलग-अलग ऐप चलाने होंगे: एक ग्रिड गेटवे और एक या अधिक ग्रिड वर्कर्स। इस ट्यूटोरियल में हम ग्रिड वर्कर्स को शुरू करने के लिए [यहाँ ](https://github.com/OpenMined/PyGrid/tree/dev/app/websocket) उपलब्ध वेबसोकेट ऐप का उपयोग करेंगे।


## ग्रिड गेटवे शुरू करना


### चरण 1: रिपॉजिटरी डाउनलोड करें

```bash
git clone https://github.com/OpenMined/PyGrid/
```


### चरण 2: निर्भरताएँ डाउनलोड करें

आपको ऐप निर्भरता स्थापित करने की आवश्यकता होगी। हम लाइब्रेरी संस्करणों के साथ समस्याओं से बचने के लिए एक स्वतंत्र [conda environment](https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html) स्थापित करने की सलाह देते हैं।

आप चलाकर निर्भरता स्थापित कर सकते हैं:

```bash
cd PyGrid/gateway/
pip install -r requirements.txt
```


### चरण 3: गेटवे ऐप शुरू करें

फिर ऐप को शुरू करने के लिए बस `gateway.py` स्क्रिप्ट चलाएं। `--start_local_db` स्वचालित रूप से एक स्थानीय डेटाबेस शुरू करता है ताकि आपको स्वयं को कॉन्फ़िगर न करना पड़े

```bash
python gateway.py --start_local_db --port=<port_number>
```

यह पते पर ऐप शुरू करेगा: `http://0.0.0.0/<port_number>`.

यह जाँचने के लिए कि इस ऐप को चलाते समय आप किन अन्य तर्कों का उपयोग कर सकते हैं, चलाएँ:

```bash
python gateway.py --help
```

आइए पोर्ट `5000` पर एक ग्रिड गेटवे शुरू करें

```bash
python gateway.py --port=5000
```

महान, इसलिए यदि आपका ऐप सफलतापूर्वक शुरू हो गया है तो स्क्रिप्ट अभी भी चलनी चाहिए।

## ग्रिड वर्कर ऐप शुरू करना

### चरण 4: ग्रिड कार्यकर्ता ऐप शुरू करना

यह एक ही प्रक्रिया है जो पहले से ही भाग 1 में वर्णित है। लेकिन हम `--gateway_url` नामक ऐप शुरू करते समय एक नया तर्क जोड़ते हैं, यह ग्रिड नेटवर्क द्वारा उपयोग किए जाने वाले पते के बराबर होना चाहिए, यह "http://localhost:5000" है।


दो श्रमिकों को शुरू करते हैं:

* बॉब पोर्ट `3000` पर
* ऐलिस पोर्ट `3001` पर

```bash
python websocket_app.py --db_url=redis:///redis:6379 --id=bob --port=3000 --gateway_url=http://localhost:5000
```

```bash
python websocket_app.py --db_url=redis:///redis:6379 --id=alice --port=3001 --gateway_url=http://localhost:5000
```

हमें हमेशा ग्रिड गेटवे शुरू करने के बाद श्रमिकों को शुरू करना चाहिए !!


महान, इसलिए यदि आपका ऐप सफलतापूर्वक शुरू हो गया है तो स्क्रिप्ट अभी भी चलनी चाहिए।


### चरण 5: ग्रिड गेटवे और श्रमिकों के साथ संचार शुरू करें

गेटवे और श्रमिकों के साथ संचार शुरू करते हैं।

In [4]:
# General dependencies
import torch as th
import syft as sy
from syft.grid.public_grid import PublicGridNetwork
from syft.workers.node_client import NodeClient
hook = sy.TorchHook(th)

In [2]:
GRID_ADDRESS = 'localhost'
GRID_PORT = '5000'

gateway = PublicGridNetwork(hook,"http://" + GRID_ADDRESS + ":" + GRID_PORT)

In [5]:
# WARNING: We should use the same id and port as the one used to start the app!!!
bob = NodeClient(hook, "ws://bob:3000")

# WARNING: We should use the same id and port as the one used to start the app!!!
alice = NodeClient(hook, "ws://alice:3001")

### चरण 6: सामान्य की तरह PySyft का उपयोग करें

अब आप बस आपके द्वारा बनाए गए कार्यकर्ता का उपयोग कर सकते हैं जैसे कि आप किसी भी अन्य सामान्य PySyft कार्यकर्ता। PySyft कैसे काम करता है, इस पर अधिक जानकारी के लिए, PySyft ट्यूटोरियल देखें: https://github.com/OpenMined/PySyft/tree/master/examples/tutorials

In [6]:
x = th.tensor([1,2,3,4]).send(bob)
x

(Wrapper)>[PointerTensor | me:7311489525 -> Bob:28859851125]

In [7]:
y = x + x
y

(Wrapper)>[PointerTensor | me:89355944812 -> Bob:43871047975]

In [8]:
y.get()

tensor([2, 4, 6, 8])

### चरण 7: ग्रिड नेटवर्क पर ऑपरेशन करें

अभी तक हमने कुछ अलग नहीं किया है, लेकिन यहां जादू है: हम नेटवर्क के साथ बातचीत कर सकते हैं ताकि इसके बारे में सामान्य जानकारी को क्वेरी कर सकें।

In [9]:
x = th.tensor([1, 2, 3, 4, 5]).tag("#tensor").send(bob)

हम पूरे नेटवर्क में एक tensor की खोज कर सकते हैं, और सभी tensors को पॉइंटर्स प्राप्त कर सकते हैं।

In [10]:
gateway.search("#tensor")

{'Bob': [(Wrapper)>[PointerTensor | me:9865241336 -> Bob:2228316112]
  	Tags: #tensor 
  	Shape: torch.Size([5])]}

In [11]:
y = th.tensor([1, 2, 3, 4, 5]).tag("#tensor").send(alice)

In [12]:
gateway.search("#tensor")

{'Bob': [(Wrapper)>[PointerTensor | me:39675731340 -> Bob:2228316112]
  	Tags: #tensor 
  	Shape: torch.Size([5])],
 'Alice': [(Wrapper)>[PointerTensor | me:69033538805 -> Alice:84105580423]
  	Tags: #tensor 
  	Shape: torch.Size([5])]}